In [1]:
import requests
import pandas as pd

## Let say

Let say, GOOG.csv is my realtime data, remember, we trained using `Close`, and `Volume` data.

So every request means new daily data.

You can improve the code to bind historical data with your own database or any websocket streaming data. Imagination is your limit now.

In [2]:
df = pd.read_csv('AMD.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-05-23,12.93,13.18,12.90,13.10,13.10,44388300
1,2018-05-24,13.06,13.43,13.03,13.41,13.41,47785700
2,2018-05-25,13.40,13.72,13.36,13.54,13.54,43850100
3,2018-05-29,13.45,13.63,13.26,13.36,13.36,39578500
4,2018-05-30,13.48,13.95,13.48,13.82,13.82,58186400


In [3]:
close = df['Close'].tolist()
volume = df['Volume'].tolist()

## Check balance

In [4]:
requests.get('http://localhost:8005/balance').json()

1000

This is the initial capital we have for now, you can check [agent.ipynb](https://github.com/huseinzol05/Stock-Prediction-Models/blob/master/realtime-agent/agent.ipynb) how I defined it, or you can overwrite it.

In [5]:
requests.get('http://localhost:8005/reset?money=1000').json()

True

In [6]:
requests.get('http://localhost:8005/balance').json()

1000

## Trading

In [7]:
import json

data = json.dumps([close[0], volume[0]])
data

'[13.1, 44388300]'

Remember, my last training session was only used `Close` and `Volume`, you need to edit it to accept any kind of parameters.

In [8]:
requests.get('http://localhost:8005/trade?data='+data).json()

{'action': 'fail',
 'balance': 1000,
 'status': 'data not enough to trade',
 'timestamp': '2019-05-26 01:12:10.232885'}

Reason why you got 'data not enough to trade', because, the agent waiting another data to complete the queue, atleast same as `window_size` size.

Last time I defined `window_size` is 20, means, it only look back 20 historical data to trade.

In [9]:
requests.get('http://localhost:8005/queue').json()

[[100.0, 226240082.86238617]]

Assume now, you have 40 times new datapoints going in, you want to trade these datapoints.

In [10]:
for i in range(100):
    data = json.dumps([close[i], volume[i]])
    requested = requests.get('http://localhost:8005/trade?data=' + data).json()
    print(requested)

{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.247068'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.249505'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.251500'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.253600'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.255749'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.257700'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.259652'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough to trade', 'timestamp': '2019-05-26 01:12:10.261590'}
{'action': 'fail', 'balance': 1000, 'status': 'data not enough t

{'action': 'sell', 'balance': 984.5599970000004, 'investment': '-0.061334 %', 'status': 'sell 1 unit, price 32.590000', 'timestamp': '2019-05-26 01:12:10.655115'}
{'action': 'buy', 'balance': 953.6699980000004, 'status': 'buy 1 unit, cost 30.889999', 'timestamp': '2019-05-26 01:12:10.662295'}
{'action': 'nothing', 'balance': 953.6699980000004, 'status': 'do nothing', 'timestamp': '2019-05-26 01:12:10.668052'}
{'action': 'nothing', 'balance': 953.6699980000004, 'status': 'do nothing', 'timestamp': '2019-05-26 01:12:10.673750'}
{'action': 'nothing', 'balance': 953.6699980000004, 'status': 'do nothing', 'timestamp': '2019-05-26 01:12:10.679499'}
{'action': 'nothing', 'balance': 953.6699980000004, 'status': 'do nothing', 'timestamp': '2019-05-26 01:12:10.685268'}
{'action': 'nothing', 'balance': 953.6699980000004, 'status': 'do nothing', 'timestamp': '2019-05-26 01:12:10.691005'}
{'action': 'sell', 'balance': 980.1299970000005, 'investment': '-18.759598 %', 'status': 'sell 1 unit, price 26

Some of investments are really doing good job, you can see some more than positive 10% investment there.